In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install faster-whisper==1.0.3 flask pyngrok pydub soundfile
!apt-get -y install ffmpeg

  Using cached faster_whisper-1.0.3-py3-none-any.whl.metadata (15 kB)
  Using cached pyngrok-7.3.0-py3-none-any.whl.metadata (8.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 51.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 45.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 90.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [5]:
from pyngrok import ngrok
ngrok.set_auth_token("31JvDkD7grsF2LO5oiprZglqqLW_7meFpK9M2Em2RtaSo81o4")

In [6]:
import os, tempfile
from flask import Flask, request, jsonify
from pydub import AudioSegment
from faster_whisper import WhisperModel
from pyngrok import ngrok

# ==== Model load ====
MODEL_NAME = os.environ.get("WHISPER_MODEL", "large-v3")
COMPUTE_TYPE = os.environ.get("WHISPER_COMPUTE", "float16")
DEVICE = "cuda"
model = WhisperModel(MODEL_NAME, device=DEVICE, compute_type=COMPUTE_TYPE)

# ==== Preprocessing ====
def preprocess_to_wav_mono16k(in_path: str) -> str:
    audio = AudioSegment.from_file(in_path)
    audio = audio.set_channels(1).set_frame_rate(16000)
    out = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    audio.export(out.name, format="wav")
    return out.name

# ==== Transcription ====
def transcribe_file_simple(in_path: str, language="en"):
    wav = preprocess_to_wav_mono16k(in_path)
    segments, info = model.transcribe(
        wav,
        language=language,
        vad_filter=True,
        beam_size=5,
        temperature=0
    )
    out = [{"start": float(s.start), "end": float(s.end), "text": s.text} for s in segments]
return {"text": " ".join(s["text"] for s in out).strip(), "segments": out}

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

RuntimeError: CUDA failed with error CUDA driver version is insufficient for CUDA runtime version

In [7]:
# ==== Flask app ====
app = Flask(__name__)

@app.route("/transcribe", methods=["POST"])
def transcribe_endpoint():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400
    file = request.files["file"]
    with tempfile.NamedTemporaryFile(suffix=os.path.splitext(file.filename)[1], delete=False) as tmp:
        file.save(tmp.name)
        result = transcribe_file_simple(tmp.name)
    return jsonify(result)

# ==== ngrok tunnel ====
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")
        
if __name__ == "__main__":
    app.run(port=5000)

Public URL: NgrokTunnel: "https://a0c2208a8663.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
